# Global fit of the model on all the lenslets
To obtain a continuous spatial covering of an image, the focal plane is divided into many different cells by the lenslet array. The signal from each pixel is then sent to a spectrograph, so that in the end every pixel generate one spectre, giving one position. Each spectre is a sum of gaussian spots.

The inverse problem approach allows to reduce errors on the raw data, which will in turn allows a better data reduction. We start with a direct model following the equation m = ∑ α g(θ,σ), with m the unknown model, α the amplitude of the gaussian spots g at the posotion θ and for a width at half maximum σ. To obtain the projection operator, we inverse the model: instead of having the position x(λ) on the detector, we want the wavelengths λ(x).

Depending on the mode (YJ or YJH), a lenslet has 3 or 4 spots. Here, we are initialising the model of these spots.
We need to compute the amplitudes a, full widths at half maximum fwhm and positions of the spots in the lenslets, and adjust the polynomial laws with cx the coefficients on x axis and cy on y axis. 

In [9]:
#using Pkg;
#Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

In [12]:
#Packages
using PIC

using Plots, StatsBase, Statistics, TwoDimensional, OptimPackNextGen, DelimitedFiles
using FITSIO, ProgressMeter
using DataFrames, CSV

plotly()                                 # Backend of Plots package

Plots.PlotlyBackend()

## Wavelengths
lasers are placed in front of the lenslets, in order to calibrate over different bands.
lasers are distributed on the detector following the dispersion law.
3 or 4 lasers are used depending on which mode is used.

In [ ]:
#Enter url from data file here:
#download(url::AbstractString, [path::AbstractString = tempname()]) -> path

In [13]:
λ1 = 987.72e-9                            # laser 1 
λ2 = 1123.71e-9                           # laser 2 
λ3 = 1309.37e-9                           # laser 3
λ4 = 1545.10e-9                           # laser 4  
λlaser = [λ1,λ2,λ3,λ4]
nλ = length(λlaser)                       # number of spots in the model
λ0 = mean(λlaser);                        # reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Calibration

x(λ) = cx0 + cx1((λ-λ0)/λ0) + cx2((λ-λ0)/λ0)²

y(λ) = cy0 + cy1((λ-λ0)/λ0) + cy2((λ-λ0)/λ0)²

c0: order 0 coefficient, corresponding to the center of the lenslet, 0 = 1025x1025 pixel, so we have to add 1025 to cx0 and cy0.

λ is in meter here, but in the previous Matlab code, λ was in micrometer. c1 and c2 are then normalised and converted.

In [14]:
coeffx = readdlm("/home/user/stage/data_HR8799_YJH/Calibration/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1] .+ 1025;                # cx0
mcx1 = median(coeffx[:,2])*λ0*1e6;        # median of cx1
mcx2 = median(coeffx[:,3])*(λ0*1e6)^2;    # median of cx2
mcx3 = median(coeffx[:,4])*(λ0*1e6)^3;    # median of cx3

coeffy = readdlm("/home/user/stage/data_HR8799_YJH/Calibration/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1].+ 1025;                 # cy0
mcy1 = median(coeffy[:,2])*λ0*1e6;        # median of cy1
mcy2 = median(coeffy[:,3])*(λ0*1e6)^2;    # median of cy2
mcy3 = median(coeffx[:,4])*(λ0*1e6)^3;    # median of cy3

## Data_processed

In [15]:
lampData =  read(FITS("/home/user/stage/data_HR8799_YJH/Data_processed/IFS_calib_spec_corrected.fits")[1]);
laserData =  read(FITS("/home/user/stage/data_HR8799_YJH/Data_processed/IFS_calib_wave_corrected.fits")[1]);
#bad pixel map, 0 or 1 value: used as precision map
badpix = Float64.(read(FITS("/home/user/stage/data_HR8799_YJH/Data_processed/IFS_BP_corrected.fits")[1])); 

## Constants

In [16]:
lensletnumber= length(cx0) # Total number of lenslet in the image

#Initialisation
#ainit = [990. , 690. , 310. , 100.];
#fwhminit = [2.3, 2.4 , 2.7, 2. ];
ainit= rand(Float64,4);
fwhminit= rand(Float64,4)*10;

#BoundingBox
largeur = 4;
hauteur = 44;
dxmin = 2;
dxmax = 2;
dymin = 21;
dymax = 18;

In [17]:
#Valid lenslet framed by bbox 
valid = (round.(cx0 .- dxmin).>0) .&  (round.(cx0 .+ dxmax).<2048) .&  
(round.(cy0 .- dymin).>0) .&  (round.(cy0 .+ dymax).<2048);

## Initialisation of the tables of results

In [18]:
lenslettab = Array{Union{LensletModel,Missing}}(missing,lensletnumber);
atab = Array{Union{Float64,Missing}}(missing,4,lensletnumber);
fwhmtab = Array{Union{Float64,Missing}}(missing,4,lensletnumber);
ctab = Array{Union{Float64,Missing}}(missing,2,4,lensletnumber);

## Fit over all the lenslets

In [19]:
#Progress bar of the fit
p = Progress(lensletnumber; showspeed=true)

Threads.@threads for i in findall(valid) # i indice des lenslet valides
    #Initialisation
    bbox = round(Int, BoundingBox(cx0[i,1]-dxmin, cx0[i,1]+dxmax, cy0[i,1]-dymin, cy0[i,1]+dymax));
    lenslettab[i] = LensletModel(λ0,nλ-1, bbox); #Contains the model for each lenslet
    Cinit= [ [cx0[i,1] mcx1 mcx2 mcx3]; [cy0[i,1] mcy1 mcy2 mcy3] ];
    xinit = vcat([fwhminit[:],Cinit[:]]...);
    #Optimisation
    laserDataView = view(laserData, bbox);
    badpixview = view(badpix,bbox);
    lkl = LikelihoodIFS(lenslettab[i],λlaser,laserDataView,badpixview);
    cost(x::Vector{Float64}) = lkl(x)
    try #The try/catch statement allows for Exceptions to be tested for
        xopt = vmlmb(cost, xinit; verb=false,ftol = (0.0,1e-8),maxeval=500);
        #Backup of the parameters into vectors
        (fwhmopt,copt) = (xopt[1:(nλ)],reshape(xopt[(nλ+1):(3*nλ)],2,:));
        atab[:,i] = lkl.amplitude;
        fwhmtab[:,i] = fwhmopt
        ctab[:,:,i] = copt
    catch
        continue
    end
    next!(p)
end
ProgressMeter.finish!(p)

Progress: 100%|███████████████████████████| Time: 2:44:05 ( 0.52  s/it)mt)m


## Backup as CVS files

In [21]:
#Tables of results with bad fit, marked as "missing"
CSV.write("amptotYJH.csv",  DataFrame(atab, :auto), missingstring="missing");
CSV.write("fwhmtotYJH.csv",  DataFrame(fwhmtab, :auto), missingstring="missing");
#separation de ctab en cxtab et cytab
cxtot = reshape(ctab[:,:,:],2,: )[1,:,:];
cytot = reshape(ctab[:,:,:],2,: )[2,:,:];  
CSV.write("cxtotYJH.csv",  DataFrame(reshape(cxtot,4,:), :auto), missingstring="missing");
CSV.write("cytotYJH.csv",  DataFrame(reshape(cytot,4,:), :auto), missingstring="missing");
CSV.write("lensletmodelYJH.csv", DataFrame([lenslettab], :auto), missingstring="missing");

#Tables of results without bad fit
atab_valid = collect(skipmissing(atab));
fwhmtab_valid = collect(skipmissing(fwhmtab));
ctab_valid = collect(skipmissing(ctab));
cxtab = reshape(ctab_valid[:,1,1],2,: )[1,:,:];
cytab = reshape(ctab_valid[:,1,1],2,: )[2,:,:];
CSV.write("atabYJH.csv",  DataFrame(reshape(atab_valid, 4 , : ), :auto));
CSV.write("fwhmtabYJH.csv",  DataFrame(reshape(fwhmtab_valid, 4, :), :auto));
CSV.write("cxtabYJH.csv",  DataFrame(reshape(cxtab,4,:), :auto));
CSV.write("cytabYJH.csv",  DataFrame(reshape(cytab,4,:), :auto));